In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"diwakarmogaralaslu","key":"01f39cff4e7593382e52958c52990706"}'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other


In [6]:
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [7]:
data = pd.read_csv("tmdb_5000_movies.csv")
data = data[data["overview"]!=" "]

In [8]:
del data["homepage"]
del data["tagline"]
del data["runtime"]

In [9]:
data.isnull().sum()

,0
budget,0
genres,0
id,0
keywords,0
original_language,0
original_title,0
overview,3
popularity,0
production_companies,0
production_countries,0


In [10]:
data = data.dropna()

In [11]:
sentence=""
for i in data.iloc[4118]["genres"].replace(" ",''):
    if i.isalpha()== True:

            sentence +=i


sentence.split("idname")[1:]

['Drama', 'Comedy', 'Music']

In [12]:
data = data[(data["genres"] != "[]") | (data["genres"] != "")]

In [13]:
data["category"] = ""

In [14]:
for i in data.index: # Use data.index to iterate through actual indices
    sentence = ""
    for j in data.loc[i, "genres"].replace(" ", ''):
        if j.isalpha():
            sentence += j
    sentence = sentence.replace('idname', " ").strip().split(" ")
    data.at[i, "category"] = sentence  # Assign the list to the cell

In [15]:
data = data.reset_index()
del data['index']

In [16]:
for i in data.index:
    sentence = ""
    if data.iloc[i]["genres"] == "[]":
      sentence = [""]
      data.at[i,"category"] = sentence
      continue

    elif data.iloc[i]["genres"] == "":
      sentence = [""]
      data.at[i,"category"] = sentence
      print(i)
      continue
    else:
        for j in data.iloc[i]["genres"].replace(" ",''):
          if j.isalpha()== True:

            sentence +=j
    sentence = sentence.replace('idname'," ").strip().split(" ")
    data.at[i,"category"] = sentence

In [17]:
data

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,status,title,vote_average,vote_count,category
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Avatar,7.2,11800,"[Action, Adventure, Fantasy, ScienceFiction]"
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Pirates of the Caribbean: At World's End,6.9,4500,"[Adventure, Fantasy, Action]"
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,Spectre,6.3,4466,"[Action, Adventure, Crime]"
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Dark Knight Rises,7.6,9106,"[Action, Crime, Drama, Thriller]"
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,John Carter,6.1,2124,"[Action, Adventure, ScienceFiction]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,El Mariachi,6.6,238,"[Action, Crime, Thriller]"
4794,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,[],Released,Newlyweds,5.9,5,"[Comedy, Romance]"
4795,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Signed, Sealed, Delivered",7.0,6,"[Comedy, Drama, Romance, TVMovie]"
4796,0,[],126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": 

In [18]:
data.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,spoken_languages,status,title,vote_average,vote_count,category
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Avatar,7.2,11800,"[Action, Adventure, Fantasy, ScienceFiction]"
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Pirates of the Caribbean: At World's End,6.9,4500,"[Adventure, Fantasy, Action]"
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,Spectre,6.3,4466,"[Action, Adventure, Crime]"
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Dark Knight Rises,7.6,9106,"[Action, Crime, Drama, Thriller]"
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,John Carter,6.1,2124,"[Action, Adventure, ScienceFiction]"


In [19]:
dam = []
for i in range(len(data)):
    for j in data.loc[i,"category"]:
        dam.append(j)

In [20]:
dam = set(dam)
dam

{'',
 'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'ScienceFiction',
 'TVMovie',
 'Thriller',
 'War',
 'Western'}

In [21]:
for i in dam:
    if i != "":
        data[i] =  0
dam = list(dam)
dam.pop(0)

''

In [22]:
dam

['Drama',
 'TVMovie',
 'Mystery',
 'Adventure',
 'Western',
 'Thriller',
 'Family',
 'Foreign',
 'Fantasy',
 'Music',
 'ScienceFiction',
 'Horror',
 'History',
 'Romance',
 'War',
 'Comedy',
 'Animation',
 'Crime',
 'Documentary',
 'Action']

In [23]:
for i in dam:
  for j in range(len(data)):
    if i in data.loc[j,"category"]:
              data.at[j,i] = 1


In [24]:
data.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'spoken_languages',
       'status', 'title', 'vote_average', 'vote_count', 'category', 'Drama',
       'TVMovie', 'Mystery', 'Adventure', 'Western', 'Thriller', 'Family',
       'Foreign', 'Fantasy', 'Music', 'ScienceFiction', 'Horror', 'History',
       'Romance', 'War', 'Comedy', 'Animation', 'Crime', 'Documentary',
       'Action'],
      dtype='object')

In [25]:
columns_2_delete = ['budget', 'genres', 'id', 'keywords','original_title','popularity', 'production_companies','release_date', 'revenue','spoken_languages', 'status', 'title', 'vote_average','vote_count',]
for column in columns_2_delete:
    del data[column]

In [26]:
import regex as re
def extract_country(s):
    if isinstance(s, str):
        match = re.search(r'"name"\s*:\s*"([^"]+)"', s)
        return match.group(1) if match else None
    else:
        return None

data["country_name"] = data["production_countries"].apply(extract_country)

In [27]:
data = data[['original_language', 'overview','country_name', 'category', 'Comedy', 'Fantasy', 'Adventure', 'Thriller', 'Mystery','Animation', 'History', 'Romance', 'Foreign', 'Horror', 'Documentary','Family', 'Western', 'Music', 'ScienceFiction', 'War', 'TVMovie','Action', 'Drama', 'Crime', ]]
data.dropna()

,original_language,overview,country_name,category,Comedy,Fantasy,Adventure,Thriller,Mystery,Animation,...,Documentary,Family,Western,Music,ScienceFiction,War,TVMovie,Action,Drama,Crime
0,en,"In the 22nd century, a paraplegic Marine is di...",United States of America,"[Action, Adventure, Fantasy, ScienceFiction]",0,1,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,en,"Captain Barbossa, long believed to be dead, ha...",United States of America,"[Adventure, Fantasy, Action]",0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,en,A cryptic message from Bond’s past sends him o...,United Kingdom,"[Action, Adventure, Crime]",0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,en,Following the death of District Attorney Harve...,United States of America,"[Action, Crime, Drama, Thriller]",0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,1
4,en,"John Carter is a war-weary, former military ca...",United States of America,"[Action, Adventure, ScienceFiction]",0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,en,Friends/fledgling entrepreneurs invent a devic...,United States of America,"[ScienceFiction, Drama, Thriller]",0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4793,es,El Mariachi just wants to play his guitar and ...,Mexico,"[Action, Crime, Thriller]",0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
4795,en,"""Signed, Sealed, Delivered"" introduces a dedic...",United States of America,"[Comedy, Drama, Romance, TVMovie]",1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4796,en,When ambitious New York attorney Sam is sent t...,United States of America,[],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
x = data[["overview","country_name","original_language"]]
y = data.iloc[:,4:]

In [43]:
y

,Comedy,Fantasy,Adventure,Thriller,Mystery,Animation,History,Romance,Foreign,Horror,Documentary,Family,Western,Music,ScienceFiction,War,TVMovie,Action,Drama,Crime
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4794,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4795,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
4796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
x.head()

,overview,country_name,original_language
0,"In the 22nd century, a paraplegic Marine is di...",United States of America,en
1,"Captain Barbossa, long believed to be dead, ha...",United States of America,en
2,A cryptic message from Bond’s past sends him o...,United Kingdom,en
3,Following the death of District Attorney Harve...,United States of America,en
4,"John Carter is a war-weary, former military ca...",United States of America,en


In [34]:
# prompt: in overview remove stopwords and do lemmatization and later use tfidf to convert overview. if possible add word parts of speech tagging

import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)

    # POS tagging (optional, but helpful for lemmatization)
    pos_tags = pos_tag(tokens)

    # Remove stopwords and non-alphanumeric characters
    processed_tokens = []
    for token, tag in pos_tags:
      token = token.lower()
      if token not in stop_words and token.isalnum():
          processed_tokens.append(token)

    # Lemmatization
    lemmatized_tokens = []
    for token, tag in pos_tags:
      if token in processed_tokens:
        pos = get_wordnet_pos(tag)
        lemmatized_tokens.append(lemmatizer.lemmatize(token, pos=pos))

    return " ".join(lemmatized_tokens)


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN


x['processed_overview'] = x['overview'].apply(preprocess_text)

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(x['processed_overview'])

# Now you have the TF-IDF matrix
X_tfidf.shape


<ipython-input-34-b3a27c2cd1f1>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['processed_overview'] = x['overview'].apply(preprocess_text)


(4798, 10832)

In [185]:

import numpy as np
import scipy

vectorizer = TfidfVectorizer()
x['overview'] = x['overview'].fillna('')
vectorizer.fit(x['overview'])
X_tfidf = vectorizer.fit_transform(x['overview'])

<ipython-input-185-5e82c305251f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['overview'] = x['overview'].fillna('')


In [208]:
!pip install --upgrade scikit-learn

In [35]:
from sklearn.linear_model import LogisticRegression
models = {}
for col in y.columns:
    model = LogisticRegression()
    model.fit(X_tfidf, y[col])
    models[col] = model
    print(col)
# Predict all columns using the trained models
predictions = pd.DataFrame()
for col, model in models.items():
    predictions[col] = model.predict(X_tfidf)

# Now 'predictions' contains the predicted values for all columns in y based on X_tfidf
predictions


Comedy
Fantasy
Adventure
Thriller
Mystery
Animation
History
Romance
Foreign
Horror
Documentary
Family
Western
Music
ScienceFiction
War
TVMovie
Action
Drama
Crime


,Comedy,Fantasy,Adventure,Thriller,Mystery,Animation,History,Romance,Foreign,Horror,Documentary,Family,Western,Music,ScienceFiction,War,TVMovie,Action,Drama,Crime
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4793,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4794,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4795,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
# prompt: now check the accuracy of y and predictions using sklearn metrics accuracy score

from sklearn.metrics import accuracy_score

# Assuming 'y' and 'predictions' are defined as in the previous code

accuracy_scores = {}
for col in y.columns:
    accuracy = accuracy_score(y[col], predictions[col])
    accuracy_scores[col] = accuracy
    print(f"Accuracy for {col}: {accuracy}")


Accuracy for Comedy: 0.8526469362234265
Accuracy for Fantasy: 0.9166319299708212
Accuracy for Adventure: 0.8599416423509796
Accuracy for Thriller: 0.8278449353897457
Accuracy for Mystery: 0.9280950395998333
Accuracy for Animation: 0.9512296790329304
Accuracy for History: 0.9589412255106294
Accuracy for Romance: 0.862025844101709
Accuracy for Foreign: 0.9929137140475198
Accuracy for Horror: 0.9045435598165903
Accuracy for Documentary: 0.9774906210921217
Accuracy for Family: 0.8995414756148395
Accuracy for Western: 0.9829095456440183
Accuracy for Music: 0.9614422676115048
Accuracy for ScienceFiction: 0.9066277615673197
Accuracy for War: 0.9699874947894956
Accuracy for TVMovie: 0.9983326385994165
Accuracy for Action: 0.8601500625260525
Accuracy for Drama: 0.8634847853272197
Accuracy for Crime: 0.8776573572321801


In [37]:
sum  = 0
for i in accuracy_scores.values():
  sum+=i
(sum/20)*100

91.76219258024176